## Load necessary libraries and paths

In [ ]:
import os
import sys
sys.path.append('../../swdb_2019_tools')
import spikeutils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.ndimage.filters import gaussian_filter1d
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys import ecephys_session
%matplotlib inline

# fix slow autocomplete
%config Completer.use_jedi = False

import platform
platstring = platform.platform()

if 'Darwin' in platstring:
    # OS X 
    data_root = "/Volumes/Brain2019/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn1' in platstring):
    # then on AWS
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2019/"

manifest_path = os.path.join(data_root, "dynamic-brain-workshop/visual_coding_neuropixels/2019/manifest.json")

## Get Session with associated units, spike times, etc.
> #### *areas*: select desired brain region acronyms to extract sessions that have cells in those regions
> #### *session*: select desired session
> #### Note: If you would like any further session/unit/spike sub-selection, do it in this chunk to feed into the next chunk

In [ ]:
cache = EcephysProjectCache.fixed(manifest=manifest_path)
sessions = cache.get_sessions()

areas = ['CA','DG']
area_sessions = [] # Adds all the sessions that have the desired areas (designated above)
for i in np.arange(len(sessions.structure_acronyms)):
    sessionid = sessions.structure_acronyms.index[i]
    if any(elem in sessions.structure_acronyms[sessionid] for elem in areas):
        area_sessions.append(sessionid)

session = cache.get_session_data(area_sessions[0])
units = session.units
spike_times = session.spike_times
sampling_rate = 30000 # The sampling rate is very stable across probes

# Just CA units
CA_units = units[units.structure_acronym =='CA']
CA_spikes = {unit : spike_times[unit] for unit in CA_units.index}

## Bin spikes by a specified sampling rate
> #### *unitsForBinning*: which units to use (should be in the form of a DF as __units__ above)
> #### *spikesForBinning*: which spikes to use (should be in the form of a dictionary as __spikes__ above)
> #### *startTime & endTime*: the desired range of time in seconds
> #### *binsize*: the desired sampling rate after binning (in Hz)

In [ ]:
unitsForBinning = CA_units
spikesForBinning = CA_spikes
unitNames = np.asarray([n for n in spikesForBinning.keys()])
toBin = [sp for sp in spikesForBinning.values()]
startTime = 0
endTime = 9000
binsize = 1000
binned = spikeutils.spiketimes_to_2D_rates(toBin, startime=startTime, stoptime=endTime, binsize=binsize)

## Creates DataFrame of RSUs and FSUs from above units

In [ ]:
cellType = []
for i in unitNames:
    if unitsForBinning[unitsForBinning.index.values == i].waveform_duration.values[0] < 0.4:
        cellType = np.append(cellType, 'FSU')
    else:
        cellType = np.append(cellType, 'RSU')
cellTypeDF = pd.DataFrame(data={'CellType': cellType}, index=unitNames.astype(int))

# Smooth firing rates with a gaussian filter
> #### Set *sigma*: the amount by which you would like the spikes to be smoothed
> #### Note: I would play around with the sigma value to get the amount of smoothing you desire for your desired application

In [ ]:
#%matplotlib notebook
# Remove above line if running in jupyter-lab
sigma = 10
smoothed = np.zeros(binned.shape)
for cell in range(binned.shape[0]):
    smoothed[cell,:] = gaussian_filter1d(binned[cell,:], sigma=sigma) # Smoothed firing rate signal

# Plot before and after smoothing
fig, ax = plt.subplots(nrows=2,ncols=1,figsize=(18,10), sharex=True, sharey=True)
for i in range(5):
    maxVal = np.max([smoothed.max(), binned.max()])
    ax[0].plot(binned[i,:] + i*maxVal/20)
    ax[0].set_title('Before Smoothing', fontsize=20)
    ax[1].plot(smoothed[i,:] + i*maxVal/20)
    ax[1].set_title('After Smoothing', fontsize=20)

## Normalize firing rates to max of 1

In [ ]:
%matplotlib inline
normalized = smoothed.copy()
for cell in range(smoothed.shape[0]):
    maxVal = np.max(smoothed[cell,:])
    if maxVal != 0:
        normalized[cell,:] = smoothed[cell,:] / maxVal

plt.figure(figsize=(20,10))
for cell in range(10):
    plt.plot(normalized[cell,:]+cell*2)
plt.show()

plt.figure(figsize=(20,5))
plt.imshow(normalized, aspect='auto', cmap='viridis')
plt.colorbar()
plt.show()

## Save binned and smoothed firing rates to CSV
> #### Insert __description__ as a string for use in output file name
> #### Insert which data to output in __traces__:
- #### *binned*: Firing rates that have been binned but not smoothed
- #### *smoothed*: Firing rates that have been both binned and smoothed
- #### *normalized*: Firing rates that have been binned, smoothed, and normalized to 1

In [ ]:
description = "CA_units"
traces = normalized
unitsToOutput = pd.concat([cellTypeDF, pd.DataFrame(normalized, index=unitNames.astype(int))], axis=1)
unitsToOutput.to_csv(('./' + description + '_session_' + str(area_sessions[0]) + "_binned" +
                      str(startTime) + "to" + str(endTime) + "Secs" + str(binsize) + "binsize" + '_GaussianSmoothed_sigma' + str(sigma) + '.csv'))